In [2]:
import pandas as pd
import os
import glob
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
pd.options.mode.chained_assignment = None
from scipy import stats
from tqdm import tqdm
sns.set_theme()
sns.set(font_scale=1)

EPSILON = 0.001
PHYS_PROPERTY = {'equate_1':'AD-controlled', 'equate_2': 'TS-controlled', 'equate_3': 'CH-controlled'}
PHYS_PROPERTY_TO_NUM = {'Average Diameter':1, 'Total Surface Area':2, 'Convex Hull': 3}
NUM_TO_PHYS_PROPERTY = {1: 'AD-controlled', 2: 'TS-controlled', 3: 'CH-controlled'}
EXPERIMENTS = ['size', 'count', 'size-count', 'count-size', 'colors', 'colors-count']
CONGRUENT_COLUMNS = ['Ratio 50 Congruent Validation Accuracy',
                     'Ratio 56 Congruent Validation Accuracy',
                     'Ratio 63 Congruent Validation Accuracy',
                     'Ratio 71 Congruent Validation Accuracy',
                     'Ratio 75 Congruent Validation Accuracy',
                     'Ratio 86 Congruent Validation Accuracy']
INCONGRUENT_COLUMNS = ['Ratio 50 Incongruent Validation Accuracy',
                     'Ratio 56 Incongruent Validation Accuracy',
                     'Ratio 63 Incongruent Validation Accuracy',
                     'Ratio 71 Incongruent Validation Accuracy',
                     'Ratio 75 Incongruent Validation Accuracy',
                     'Ratio 86 Incongruent Validation Accuracy']
TO_REMOVE = 9999999.9
STD_FILTER = 2.5
CONGRUENT_VALUE = 1
INCONGRUENT_VALUE = 0
CONGRUENCY = {CONGRUENT_VALUE: "Congruent", INCONGRUENT_VALUE: "Incongruent"}
RATIOS = ['50','56','63','71','75','86']

In [3]:
def split_train_test_results(files_names):
    tests = []
    for file in files_names:
        if "Tested_on" in file:
            tests.append(file)
        else:
            train = file
    return tests, train

Train and test result files as a dict

Aggregate all files into dict

In [4]:
PATH = "/Users/gali.k/phd/phd_2021/results_proffeciancy_0.65_counting_0.83_batch_1024_epoch_3"
result_dict = {}
for equate in PHYS_PROPERTY.keys():
    equate_dict = {}
    for experiment in EXPERIMENTS:
        curr_results_path = PATH + os.sep + equate + os.sep + experiment
        result_file_names = glob.glob(curr_results_path + os.sep + f"Results_*.csv")
        if len(result_file_names) > 0:
            tests, training_result_file_name = split_train_test_results(result_file_names)
            testing_file_name1 = tests[0]
            testing_file_name2 = tests[1]
            training_result_df = pd.read_csv(training_result_file_name)
            testing_file_1_df =  pd.read_csv(testing_file_name1)
            testing_file_1_df['Tested_On_Equate'] = testing_file_name1[testing_file_name1.find('Tested_on'): testing_file_name1.find('AvgAccuracy')-1].replace('Tested_on_Equate', 'equate')
            testing_file_2_df =  pd.read_csv(testing_file_name2)
            testing_file_2_df['Tested_On_Equate'] = testing_file_name2[testing_file_name2.find('Tested_on'): testing_file_name2.find('AvgAccuracy')-1].replace('Tested_on_Equate', 'equate')

            equate_dict.update({experiment: { "train" : training_result_df,
                                "test_1": testing_file_1_df,
                                "test_2": testing_file_2_df}})
            result_dict.update({equate: equate_dict})

Create ANOVA CSV

In [5]:
def prepare_ratios_df(curr_df):
    ratios_df = pd.DataFrame()
    for ratio in RATIOS:
        ratio_cong_df = pd.DataFrame()
        ratio_cong_df['UNIQUE_SUBJECT_UID'] = curr_df['UNIQUE_SUBJECT_UID']
        ratio_cong_df['Task'] = curr_df['Task']
        ratio_cong_df['Generations'] = curr_df['Generations']
        ratio_cong_df['Train'] = curr_df['Equate'].apply(lambda val: NUM_TO_PHYS_PROPERTY[val])
        ratio_cong_df['Test'] = curr_df['Tested_On_Equate'].apply(lambda val: PHYS_PROPERTY[val])
        ratio_cong_df['Congruity'] = 'Congruent'
        ratio_cong_df['Ratio'] = ratio
        ratio_cong_df['Training Accuracy'] = curr_df[f'Ratio {ratio} Congruent Training Accuracy']
        ratio_cong_df['Validation Accuracy'] = curr_df[f'Ratio {ratio} Congruent Validation Accuracy']
        ratio_cong_df['Training Loss'] = curr_df[f'Ratio {ratio} Congruent Validation Loss']
        ratio_cong_df['Validation Loss'] = curr_df[f'Ratio {ratio} Congruent Validation Loss']
        ratio_cong_df['Layers'] = curr_df['Layers']
        ratio_cong_df['Neurons'] = curr_df['Nuerons']
        ratio_cong_df['Activation'] = curr_df['Activation']
        ratio_cong_df['Optimizer'] = curr_df['Optimizer']

        ratio_incong_df = pd.DataFrame()
        ratio_incong_df['UNIQUE_SUBJECT_UID'] = curr_df['UNIQUE_SUBJECT_UID']
        ratio_incong_df['Task'] = curr_df['Task']
        ratio_incong_df['Generations'] = curr_df['Generations']
        ratio_incong_df['Train'] = curr_df['Equate'].apply(lambda val: NUM_TO_PHYS_PROPERTY[val])
        ratio_incong_df['Test'] = curr_df['Tested_On_Equate'].apply(lambda val: PHYS_PROPERTY[val])
        ratio_incong_df['Congruity'] = 'Incongruent'
        ratio_incong_df['Ratio'] = ratio
        ratio_incong_df['Training Accuracy'] = curr_df[f'Ratio {ratio} Incongruent Training Accuracy']
        ratio_incong_df['Validation Accuracy'] = curr_df[f'Ratio {ratio} Incongruent Validation Accuracy']
        ratio_incong_df['Training Loss'] = curr_df[f'Ratio {ratio} Incongruent Validation Loss']
        ratio_incong_df['Validation Loss'] = curr_df[f'Ratio {ratio} Incongruent Validation Loss']
        ratio_incong_df['Layers'] = curr_df['Layers']
        ratio_incong_df['Neurons'] = curr_df['Nuerons']
        ratio_incong_df['Activation'] = curr_df['Activation']
        ratio_incong_df['Optimizer'] = curr_df['Optimizer']

        ratio_df = pd.concat([ratio_cong_df, ratio_incong_df])
        ratios_df = pd.concat([ratios_df, ratio_df])
    return ratios_df.reset_index(drop=True)

In [6]:
result_dict.keys()
anova_df = pd.DataFrame()
mixed_anova_df = pd.DataFrame()
for phys_prop in tqdm(result_dict.keys()):
    #print(f"Working on {phys_prop}")
    exp_result = result_dict[phys_prop]
    for task in exp_result.keys():
        #print(f"Task: {task}")
        train_df = exp_result[task]['train']
        last_gen = train_df['Generations'].max()
        train_df = train_df.query(f"Generations == {last_gen}")
        train_df['Tested_On_Equate'] = phys_prop
        train_df['UNIQUE_SUBJECT_UID'] = train_df['Subject_UID'].apply(lambda x: task + "_" + phys_prop + "_" + str(x))
        train_ratios_df = prepare_ratios_df(train_df)

        test1_df = exp_result[task]['test_1']
        test1_df['UNIQUE_SUBJECT_UID'] = test1_df['Subject_UID'].apply(lambda x: task + "_" + phys_prop + "_" + str(x))
        test_1_ratios_df = prepare_ratios_df(test1_df)

        test2_df = exp_result[task]['test_2']
        test2_df['UNIQUE_SUBJECT_UID'] = test2_df['Subject_UID'].apply(lambda x: task + "_" + phys_prop + "_" + str(x))
        test_2_ratios_df = prepare_ratios_df(test2_df)

        mixed_anova_df = pd.concat([mixed_anova_df, train_ratios_df])
        mixed_anova_df = pd.concat([mixed_anova_df, test_1_ratios_df])
        mixed_anova_df = pd.concat([mixed_anova_df, test_2_ratios_df])
mixed_anova_df.to_csv('mixed_anova_df.csv')

100%|██████████| 3/3 [00:03<00:00,  1.18s/it]


In [7]:
len(mixed_anova_df)

12960

Filter nulls

In [8]:
non_na_mixed_anova = mixed_anova_df.dropna()
len(non_na_mixed_anova)

12816

In [9]:
non_na_mixed_anova.describe()

,Generations,Training Accuracy,Validation Accuracy,Training Loss,Validation Loss,Layers
count,12816.000000,12816.000000,12816.000000,1.281600e+04,1.281600e+04,12816.000000
mean,16.862360,0.762317,0.762317,inf,inf,3.721910
std,10.774695,0.234168,0.234168,NaN,NaN,1.018781
min,3.000000,0.050000,0.050000,0.000000e+00,0.000000e+00,2.000000
25%,7.000000,0.600000,0.600000,4.211250e-01,4.211250e-01,3.000000
50%,13.000000,0.850000,0.850000,6.035000e-01,6.035000e-01,4.000000
75%,30.000000,1.000000,1.000000,7.151250e-01,7.151250e-01,5.000000
max,30.000000,1.000000,1.000000,inf,inf,5.000000


In [9]:
non_na_mixed_anova.columns

Index(['UNIQUE_SUBJECT_UID', 'Task', 'Generations', 'Train', 'Test',
       'Congruity', 'Ratio', 'Training Accuracy', 'Validation Accuracy',
       'Training Loss', 'Validation Loss', 'Layers', 'Neurons', 'Activation',
       'Optimizer'],
      dtype='object')

Filter extreme values

In [10]:
anova_df_for_R = non_na_mixed_anova[['UNIQUE_SUBJECT_UID', 'Task','Train','Test','Generations','Congruity','Ratio','Validation Accuracy']]
anova_df_for_R.head()

,UNIQUE_SUBJECT_UID,Task,Train,Test,Generations,Congruity,Ratio,Validation Accuracy
0,size_equate_1_121,size,AD-controlled,AD-controlled,7,Congruent,50,1.0
1,size_equate_1_122,size,AD-controlled,AD-controlled,7,Congruent,50,1.0
2,size_equate_1_123,size,AD-controlled,AD-controlled,7,Congruent,50,1.0
3,size_equate_1_124,size,AD-controlled,AD-controlled,7,Congruent,50,1.0
4,size_equate_1_125,size,AD-controlled,AD-controlled,7,Congruent,50,1.0


In [11]:
anova_df_for_R.to_csv("/Users/gali.k/phd/phd_2021/data_analysis/analysis/balanced_exp/anova_df_for_R_06_02_23.csv")

Total Generations data

In [11]:
anova_df_for_R_total_generations = anova_df_for_R[['UNIQUE_SUBJECT_UID', 'Task','Train', 'Generations']].groupby(['UNIQUE_SUBJECT_UID', 'Task','Train']).mean()

In [13]:
anova_df_for_R_total_generations.to_csv("/Users/gali.k/phd/phd_2021/data_analysis/analysis/balanced_exp/anova_df_for_R_06_02_23_generations.csv")

Separating to physical and numerical

In [12]:
anova_df_for_R_physical_runs = anova_df_for_R.query(f"Task == 'size' or Task == 'count-size' or Task == 'colors'")

In [13]:
anova_df_for_R_numerical_runs = anova_df_for_R.query(f"Task == 'count' or Task == 'size-count' or Task == 'colors-count'")

In [16]:
anova_df_for_R_physical_runs.to_csv("/Users/gali.k/phd/phd_2021/data_analysis/analysis/balanced_exp/anova_df_for_R_06_02_23_physical_accuracy.csv")

In [17]:
anova_df_for_R_numerical_runs.to_csv("/Users/gali.k/phd/phd_2021/data_analysis/analysis/balanced_exp/anova_df_for_R_06_02_23_numerical_accuracy.csv")

In [14]:
anova_df_for_R_numerical_runs_generations = anova_df_for_R_numerical_runs[['UNIQUE_SUBJECT_UID', 'Task','Train', 'Generations']].groupby(['UNIQUE_SUBJECT_UID', 'Task','Train']).mean()

In [15]:
anova_df_for_R_numerical_runs_generations['Generations'].unique()

array([11., 30., 13.])

In [20]:
anova_df_for_R_numerical_runs_generations.to_csv("/Users/gali.k/phd/phd_2021/data_analysis/analysis/balanced_exp/anova_df_for_R_06_02_23_numerical_generations.csv")

In [16]:
anova_df_for_R_physical_runs_generations = anova_df_for_R_physical_runs[['UNIQUE_SUBJECT_UID', 'Task','Train', 'Generations']].groupby(['UNIQUE_SUBJECT_UID', 'Task','Train']).mean()

In [22]:
anova_df_for_R_physical_runs_generations.to_csv("/Users/gali.k/phd/phd_2021/data_analysis/analysis/balanced_exp/anova_df_for_R_06_02_23_physical_generations.csv")

************* Until here for the R anova analysis! **************

In [17]:
overall_means_physical = anova_df_for_R_physical_runs[['Task','Congruity', 'Validation Accuracy']].groupby(['Task','Congruity']).mean()

In [18]:
overall_means_numerical = anova_df_for_R_numerical_runs[['Task','Congruity', 'Validation Accuracy']].groupby(['Task','Congruity']).mean()

In [19]:
overall_means_numerical.head(10)

Validation Accuracy
Task         Congruity                       
colors-count Congruent               0.786991
             Incongruent             0.569074
count        Congruent               0.849444
             Incongruent             0.558380
size-count   Congruent               0.840913
             Incongruent             0.609510

In [25]:
overall_means_numerical_with_ratio = anova_df_for_R_numerical_runs[['Task','Train', 'Test', 'Congruity', 'Validation Accuracy', 'Ratio']].groupby(['Task','Train', 'Test','Congruity','Ratio']).mean()

In [38]:
overall_means_numerical_with_ratio.query("(Task == 'size-count') and Train == 'TS-controlled' and Test == 'TS-controlled'  and Congruity == 'Incongruent'").head(100)

Validation Accuracy
Task       Train         Test          Congruity   Ratio                     
size-count TS-controlled TS-controlled Incongruent 50                0.694737
                                                   56                0.944737
                                                   63                0.834211
                                                   71                0.857895
                                                   75                0.876316
                                                   86                0.721053

In [24]:
overall_means_physical.head(10)

Validation Accuracy
Task       Congruity                       
colors     Congruent               0.862917
           Incongruent             0.882315
count-size Congruent               0.811973
           Incongruent             0.703113
size       Congruent               0.909369
           Incongruent             0.764736

In [25]:
generations_df = pd.read_csv("/Users/gali.k/phd/phd_2021/data_analysis/analysis/balanced_exp/anova_df_for_R_06_02_23_generations.csv")

In [26]:
generations_df.drop(columns="Unnamed: 0", inplace=True)

KeyError: "['Unnamed: 0'] not found in axis"

In [27]:
generations_df.head(10)

,UNIQUE_SUBJECT_UID,Task,Train,Generations
0,colors-count_equate_1_201,colors-count,AD-controlled,11.0
1,colors-count_equate_1_202,colors-count,AD-controlled,11.0
2,colors-count_equate_1_203,colors-count,AD-controlled,11.0
3,colors-count_equate_1_204,colors-count,AD-controlled,11.0
4,colors-count_equate_1_205,colors-count,AD-controlled,11.0
5,colors-count_equate_1_206,colors-count,AD-controlled,11.0
6,colors-count_equate_1_207,colors-count,AD-controlled,11.0
7,colors-count_equate_1_208,colors-count,AD-controlled,11.0
8,colors-count_equate_1_209,colors-count,AD-controlled,11.0
9,colors-count_equate_1_210,colors-count,AD-controlled,11.0
